In [94]:
import sklearn
import os
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split

**Importing Datasets**

**players_21 and players_22**

In [95]:
p21 = pd.read_csv('players_21.csv')
p22 = pd.read_csv('players_22.csv')

C:\Users\skabs\AppData\Local\Temp\ipykernel_23292\3417458140.py:2: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  p22 = pd.read_csv('players_22.csv')


**Checking for NAN Values**

In [96]:
p21.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 110 columns):
 #    Column                       Dtype  
---   ------                       -----  
 0    sofifa_id                    int64  
 1    player_url                   object 
 2    short_name                   object 
 3    long_name                    object 
 4    player_positions             object 
 5    overall                      int64  
 6    potential                    int64  
 7    value_eur                    float64
 8    wage_eur                     float64
 9    age                          int64  
 10   dob                          object 
 11   height_cm                    int64  
 12   weight_kg                    int64  
 13   club_team_id                 float64
 14   club_name                    object 
 15   league_name                  object 
 16   league_level                 float64
 17   club_position                object 
 18   club_jersey_number      

In [97]:
useless_columns = ['sofifa_id', 'player_url','short_name','long_name','player_positions','dob','club_name','league_name','club_loaned_from','club_joined','club_position','nationality_name','nation_position','nation_team_id','nation_jersey_number','body_type','real_face','player_tags','player_traits','player_face_url','club_logo_url','club_flag_url','nation_logo_url','nation_flag_url']

In [98]:
p21.drop(useless_columns, axis = 1, inplace=True)

In [99]:
p21.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 86 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      18944 non-null  int64  
 1   potential                    18944 non-null  int64  
 2   value_eur                    18707 non-null  float64
 3   wage_eur                     18719 non-null  float64
 4   age                          18944 non-null  int64  
 5   height_cm                    18944 non-null  int64  
 6   weight_kg                    18944 non-null  int64  
 7   club_team_id                 18719 non-null  float64
 8   league_level                 18719 non-null  float64
 9   club_jersey_number           18719 non-null  float64
 10  club_contract_valid_until    18719 non-null  float64
 11  nationality_id               18944 non-null  int64  
 12  preferred_foot               18944 non-null  object 
 13  weak_foot       

**Seperating the object columns from the dataset**

In [100]:
p21_object_columns = p21.select_dtypes(include= ['object'])

**Seperating the numeric columns from the dataset**

In [101]:
p21_numeric_columns = p21.select_dtypes(include= ['int', 'int64', 'float'])

In [102]:
p21_position_values = p21_object_columns.drop(['preferred_foot','work_rate'], axis = 1)
p21_object_columns.drop(p21_position_values.columns, axis = 1, inplace=True)

In [103]:
p21_position_values

,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,89+3,89+3,89+3,92,93,93,93,92,93,93,...,65+3,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3,19+3
1,91+1,91+1,91+1,89,91,91,91,89,88+3,88+3,...,61+3,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3,20+3
2,89+2,89+2,89+2,85,87,87,87,85,85+3,85+3,...,65+3,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3
3,84+3,84+3,84+3,90,89,89,89,90,90+1,90+1,...,62+3,62+3,62+3,67+3,62+3,49+3,49+3,49+3,62+3,20+3
4,83+3,83+3,83+3,88,88,88,88,88,89+2,89+2,...,80+3,80+3,80+3,79+3,75+3,69+3,69+3,69+3,75+3,21+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,32+2,32+2,32+2,30,30,30,30,30,29+2,29+2,...,41+2,41+2,41+2,42+2,45+2,47+2,47+2,47+2,45+2,15+2
18940,41+2,41+2,41+2,45,44,44,44,45,46+2,46+2,...,48+2,48+2,48+2,47+2,47+2,46+2,46+2,46+2,47+2,15+2
18941,46+1,46+1,46+1,47,46,46,46,47,47,47,...,47,47,47,47,47,46+1,46+1,46+1,47,11+2
18942,47+2,47+2,47+2,49,49,49,49,49,49+2,49+2,...,32+2,32+2,32+2,35+2,33+2,26+2,26+2,26+2,33+2,14+2


**Handling positions_values column**

In [104]:
def remove_modifiers(s):
    if '+' in s:
        return s.split('+')[0]
    if '-' in s:
        return s.split('-')[0]
    return s

# Apply the custom function to all columns using applymap
p21_position_values = p21_position_values.applymap(remove_modifiers)

# Print the modified DataFrame
p21_position_values

,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,89,89,89,92,93,93,93,92,93,93,...,65,65,65,66,62,52,52,52,62,19
1,91,91,91,89,91,91,91,89,88,88,...,61,61,61,65,61,54,54,54,61,20
2,89,89,89,85,87,87,87,85,85,85,...,65,65,65,64,61,60,60,60,61,19
3,84,84,84,90,89,89,89,90,90,90,...,62,62,62,67,62,49,49,49,62,20
4,83,83,83,88,88,88,88,88,89,89,...,80,80,80,79,75,69,69,69,75,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,32,32,32,30,30,30,30,30,29,29,...,41,41,41,42,45,47,47,47,45,15
18940,41,41,41,45,44,44,44,45,46,46,...,48,48,48,47,47,46,46,46,47,15
18941,46,46,46,47,46,46,46,47,47,47,...,47,47,47,47,47,46,46,46,47,11
18942,47,47,47,49,49,49,49,49,49,49,...,32,32,32,35,33,26,26,26,33,14


In [105]:
#Converting the position values into int type
p21_position_values = p21_position_values.astype('int64')

In [106]:
#Checking the number of null values in each column
p21_position_values.isnull().sum()

ls     0
st     0
rs     0
lw     0
lf     0
cf     0
rf     0
rw     0
lam    0
cam    0
ram    0
lm     0
lcm    0
cm     0
rcm    0
rm     0
lwb    0
ldm    0
cdm    0
rdm    0
rwb    0
lb     0
lcb    0
cb     0
rcb    0
rb     0
gk     0
dtype: int64

**Handling the numeric column**

In [107]:
#Checking the percentage of nan values in the numeric column
nan_percentage = (p21_numeric_columns.isna().sum() / len(p21_numeric_columns)) * 100
nan_percentage

overall                         0.000000
potential                       0.000000
value_eur                       1.251056
wage_eur                        1.187711
age                             0.000000
height_cm                       0.000000
weight_kg                       0.000000
club_team_id                    1.187711
league_level                    1.187711
club_jersey_number              1.187711
club_contract_valid_until       1.187711
nationality_id                  0.000000
weak_foot                       0.000000
skill_moves                     0.000000
international_reputation        0.000000
release_clause_eur              5.252323
pace                           10.995566
shooting                       10.995566
passing                        10.995566
dribbling                      10.995566
defending                      10.995566
physic                         10.995566
attacking_crossing              0.000000
attacking_finishing             0.000000
attacking_headin

In [108]:
p21_numeric_columns.drop(['goalkeeping_speed'], axis = 1, inplace=True) #Removing because it has a high percentage of nan values

In [109]:
#Filling nan values by imputation with Mean/Median
p21_numeric_columns.fillna(p21_numeric_columns.mean(), inplace=True)

In [110]:
p21_numeric_columns.isna().sum()

overall                        0
potential                      0
value_eur                      0
wage_eur                       0
age                            0
height_cm                      0
weight_kg                      0
club_team_id                   0
league_level                   0
club_jersey_number             0
club_contract_valid_until      0
nationality_id                 0
weak_foot                      0
skill_moves                    0
international_reputation       0
release_clause_eur             0
pace                           0
shooting                       0
passing                        0
dribbling                      0
defending                      0
physic                         0
attacking_crossing             0
attacking_finishing            0
attacking_heading_accuracy     0
attacking_short_passing        0
attacking_volleys              0
skill_dribbling                0
skill_curve                    0
skill_fk_accuracy              0
skill_long

**Handling the object column**

In [111]:
#Checking for nan values in the object column
p21_object_columns.isna().sum()

preferred_foot    0
work_rate         0
dtype: int64

In [112]:
#One-hot encoding for 'Preferred foot' and 'Work rate'
p21_object_columns_encoded = pd.get_dummies(p21_object_columns, p21_object_columns.columns, dtype= int)

p21_object_columns_encoded.head()

,preferred_foot_Left,preferred_foot_Right,work_rate_High/High,work_rate_High/Low,work_rate_High/Medium,work_rate_Low/High,work_rate_Low/Low,work_rate_Low/Medium,work_rate_Medium/High,work_rate_Medium/Low,work_rate_Medium/Medium
0,1,0,0,0,0,0,0,0,0,1,0
1,0,1,0,1,0,0,0,0,0,0,0
2,0,1,0,0,1,0,0,0,0,0,0
3,0,1,0,0,1,0,0,0,0,0,0
4,0,1,1,0,0,0,0,0,0,0,0


**Combining the refined Object and Numeric columns**

In [113]:
#The dataframe that we will be using to train our model
p21_refined = pd.concat([p21_numeric_columns, p21_object_columns_encoded,p21_position_values],axis = 1)

p21_refined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 94 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      18944 non-null  int64  
 1   potential                    18944 non-null  int64  
 2   value_eur                    18944 non-null  float64
 3   wage_eur                     18944 non-null  float64
 4   age                          18944 non-null  int64  
 5   height_cm                    18944 non-null  int64  
 6   weight_kg                    18944 non-null  int64  
 7   club_team_id                 18944 non-null  float64
 8   league_level                 18944 non-null  float64
 9   club_jersey_number           18944 non-null  float64
 10  club_contract_valid_until    18944 non-null  float64
 11  nationality_id               18944 non-null  int64  
 12  weak_foot                    18944 non-null  int64  
 13  skill_moves     

In [114]:
#Scaling the new dataframe
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
p21_refined_scaled = scaler.fit_transform(p21_refined)
p21_refined_scaled_df = pd.DataFrame(p21_refined_scaled, columns =p21_refined.columns)
p21_refined_scaled_df

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_jersey_number,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,3.902006,3.586563,13.073165,27.857178,1.655055,-1.639556,-0.427506,-0.889589,-0.483897,-0.624556,...,0.823559,0.823559,0.823559,0.872466,0.622566,-0.019489,-0.019489,-0.019489,0.622566,-0.153779
1,3.759192,3.422893,7.809992,10.662997,2.080838,0.851107,1.131240,-0.893246,-0.483897,-0.801490,...,0.535536,0.535536,0.535536,0.799945,0.551203,0.115304,0.115304,0.115304,0.551203,-0.087867
2,3.616378,3.259222,14.047827,11.674420,1.229273,0.411579,0.706127,-0.893694,-0.483897,-0.683534,...,0.823559,0.823559,0.823559,0.727424,0.551203,0.519682,0.519682,0.519682,0.551203,-0.153779
3,3.616378,3.259222,16.776879,13.191553,0.590598,-0.907008,-0.994323,-0.892724,-0.483897,-0.624556,...,0.607542,0.607542,0.607542,0.944986,0.622566,-0.221678,-0.221678,-0.221678,0.622566,-0.087867
4,3.616378,3.259222,16.387015,18.248665,0.803490,-0.027950,-0.710915,-0.893899,-0.483897,-0.211711,...,1.903646,1.903646,1.903646,1.815235,1.550287,1.126250,1.126250,1.126250,1.550287,-0.021955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,-2.667458,-3.123941,-0.368070,-0.412078,-0.899642,-0.613988,-0.710915,1.199161,-0.483897,0.260112,...,-0.904581,-0.904581,-0.904581,-0.868033,-0.590607,-0.356471,-0.356471,-0.356471,-0.590607,-0.417428
18940,-2.667458,-2.960270,-0.368070,-0.412078,-0.899642,-1.053517,-0.994323,1.206159,-0.483897,0.967846,...,-0.400540,-0.400540,-0.400540,-0.505429,-0.447880,-0.423867,-0.423867,-0.423867,-0.447880,-0.417428
18941,-2.667458,-3.942295,-0.371319,-0.361507,0.590598,0.558088,0.564423,1.191864,-0.483897,0.731935,...,-0.472546,-0.472546,-0.472546,-0.505429,-0.447880,-0.423867,-0.423867,-0.423867,-0.447880,-0.681077
18942,-2.667458,-0.668878,-0.360272,-0.437363,-1.751207,-1.493046,-2.411365,-0.858255,3.599992,0.378068,...,-1.552633,-1.552633,-1.552633,-1.375678,-1.446964,-1.771796,-1.771796,-1.771796,-1.446964,-0.483341


In [115]:
p21_correlation = p21_refined_scaled_df.corr(numeric_only=True, method = 'spearman')

In [116]:
pd.set_option('display.max_rows', 100)  # To display all correlations
p21_correlation['overall'].sort_values()

club_team_id                  -0.271938
club_jersey_number            -0.233299
work_rate_Medium/Medium       -0.226299
league_level                  -0.158804
preferred_foot_Right          -0.050195
work_rate_Low/Low             -0.036296
work_rate_Low/Medium          -0.029711
nationality_id                -0.021208
work_rate_Medium/Low          -0.008957
goalkeeping_reflexes           0.014701
work_rate_Low/High             0.018082
goalkeeping_diving             0.018828
goalkeeping_kicking            0.021657
goalkeeping_positioning        0.025230
goalkeeping_handling           0.025393
height_cm                      0.028370
work_rate_High/Low             0.035766
preferred_foot_Left            0.050195
work_rate_Medium/High          0.090454
club_contract_valid_until      0.119447
movement_balance               0.135678
weight_kg                      0.136469
work_rate_High/High            0.138639
work_rate_High/Medium          0.138971
weak_foot                      0.210958


In [117]:
chosen_columns = ['overall', 'release_clause_eur', 'value_eur', 'movement_reactions', 'wage_eur', 'mentality_composure','passing','dribbling','potential','mentality_vision','age','shooting','mentality_positioning','power_stamina','movement_agility']

In [118]:
p21_refined_scaled_df = p21_refined_scaled_df[chosen_columns]
p21_refined_scaled_df

,overall,release_clause_eur,value_eur,movement_reactions,wage_eur,mentality_composure,passing,dribbling,potential,mentality_vision,age,shooting,mentality_positioning,power_stamina,movement_agility
0,3.902006,13.695696,13.073165,3.554438,27.857178,3.137573,3.493625,3.433013,3.586563,3.000047,1.655055,3.009606,2.198411,0.592251,1.891773
1,3.759192,7.312715,7.809992,3.664174,10.662997,3.055051,2.461857,2.800094,3.422893,2.052781,2.080838,3.085367,2.301274,1.347996,1.618265
2,3.616378,13.042079,14.047827,3.444701,11.674420,2.477402,2.152326,2.378148,3.259222,1.834181,1.229273,2.933845,2.249843,0.844166,0.934495
3,3.616378,16.565484,16.776879,3.225227,13.191553,2.890009,2.977741,3.327526,3.259222,2.635714,0.590598,2.479280,1.889820,1.159060,2.233658
4,3.616378,16.003782,16.387015,3.225227,18.248665,2.724966,3.699979,2.694607,3.259222,2.927180,0.803490,2.555040,1.941252,1.662889,1.002872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,-2.667458,-0.432955,-0.368070,-1.493455,-0.412078,-1.483625,-3.212867,-3.740069,-3.123941,-2.100615,-0.899642,-2.217897,-1.298952,-0.478388,-1.253569
18940,-2.667458,-0.431423,-0.368070,-1.273981,-0.412078,-1.896232,-0.839801,-1.630339,-2.960270,-0.060350,-0.899642,-1.536049,-0.938929,-0.919239,-0.911684
18941,-2.667458,-0.433977,-0.371319,-1.932402,-0.361507,-1.896232,-0.839801,-1.735826,-3.942295,-0.716150,0.590598,-1.157245,-0.116020,-0.541367,-0.501422
18942,-2.667458,-0.421926,-0.360272,-0.944771,-0.437363,-1.071018,-1.768392,-0.997420,-0.668878,-0.351817,-1.751207,-0.475396,-0.373179,-0.478388,0.250725


**Training the Model**

In [119]:
#Training with a VotingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

In [120]:
#Loading the data
X = p21_refined_scaled_df.drop(['overall'], axis = 1)
y = p21_refined_scaled_df['overall']

In [121]:
# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Training with VotingRegressor using LinearRegression, SVR, KNeighborsRegressor**

In [122]:
# Create individual regression models
svr = SVR()
lr = LinearRegression()
knn_regressor = KNeighborsRegressor(n_neighbors=5)  # You can adjust the number of neighbors

In [123]:
# Create a VotingRegressor using the individual models
voting_regressor = VotingRegressor(estimators=[('SupportVector', svr),
                                              ('linear_regression', lr),
                                              ('knn', knn_regressor)], n_jobs=-1)

# Train the VotingRegressor on the training data
voting_regressor.fit(X_train, y_train)

VotingRegressor(estimators=[('SupportVector', SVR()),
                            ('linear_regression', LinearRegression()),
                            ('knn', KNeighborsRegressor())],
                n_jobs=-1)

In [124]:
# Make predictions on the test set
y_pred = voting_regressor.predict(X_test)

In [125]:
from sklearn.model_selection import cross_val_score

In [126]:
#Performing cross-validation on the model
scores = cross_val_score(voting_regressor, X, y, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Calculate the mean and standard deviation of the scores
mean_score = scores.mean()
std_score = scores.std()

print("Cross-Validation Scores: ", scores)
print("Mean Score: ", mean_score)
print("Standard Deviation: ", std_score)

Cross-Validation Scores:  [-1.12324838 -0.04132489 -0.03878051 -0.04570213 -0.4644155 ]
Mean Score:  -0.3426942825366449
Standard Deviation:  0.4231955066879291


**Training using a Random Forest Regressor**

In [127]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Random Forest Regressor
RFR = RandomForestRegressor()
#Defining paramaters
PARAMETERS = {
"max_depth":[2,5, 6, 12],
"n_estimators":[100,200,500,1000]
}
model_rfr = GridSearchCV(RFR, param_grid = PARAMETERS, n_jobs = -1)

In [128]:
# Train the model on the training data
model_rfr.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model_rfr.predict(X_test)

In [154]:
model_rfr.best_estimator_

RandomForestRegressor(max_depth=12, n_estimators=1000)

In [129]:
#Performing cross-validation on the model
scores = cross_val_score(model_rfr, X, y,cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Calculate the mean and standard deviation of the scores
mean_score = scores.mean()
std_score = scores.std()

print("Cross-Validation Scores: ", scores)
print("Mean Score: ", mean_score)
print("Standard Deviation: ", std_score)

c:\Users\skabs\anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Cross-Validation Scores:  [-0.57273463 -0.05990804 -0.03267394 -0.0458276  -0.56591594]
Mean Score:  -0.2554120289597527
Standard Deviation:  0.2564628682118595


In [130]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

cv=KFold(n_splits=3)
PARAMETERS ={
"max_depth":[2,5, 6, 12],
"learning_rate":[0.5, 0.3, 0.05, 0.01],
"n_estimators":[100,200,500, 1000]}

# Create a Gradient Boosting Regressor
gbr = GradientBoostingRegressor()
model_gbr = GridSearchCV(gbr,param_grid=PARAMETERS,cv=cv)

# Train the model on the training data
model_gbr.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model_gbr.predict(X_test)

In [155]:
model_gbr.best_estimator_

GradientBoostingRegressor(learning_rate=0.05, max_depth=6, n_estimators=1000)

In [131]:
#Performing cross-validation on the model
scores = cross_val_score(model_gbr, X, y, cv=5, scoring='neg_mean_squared_error')

# Calculate the mean and standard deviation of the scores
mean_score = scores.mean()
std_score = scores.std()

print("Cross-Validation Scores: ", scores)
print("Mean Score: ", mean_score)
print("Standard Deviation: ", std_score)

KeyboardInterrupt: 

**Testing**

In [132]:
p22.drop(useless_columns, axis = 1, inplace=True)

In [133]:
p22.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 86 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      19239 non-null  int64  
 1   potential                    19239 non-null  int64  
 2   value_eur                    19165 non-null  float64
 3   wage_eur                     19178 non-null  float64
 4   age                          19239 non-null  int64  
 5   height_cm                    19239 non-null  int64  
 6   weight_kg                    19239 non-null  int64  
 7   club_team_id                 19178 non-null  float64
 8   league_level                 19178 non-null  float64
 9   club_jersey_number           19178 non-null  float64
 10  club_contract_valid_until    19178 non-null  float64
 11  nationality_id               19239 non-null  int64  
 12  preferred_foot               19239 non-null  object 
 13  weak_foot       

In [134]:
p22_object_columns = p22.select_dtypes(include= ['object'])

In [135]:
p22_numeric_columns = p22.select_dtypes(include= ['int', 'int64', 'float'])

In [136]:
p22_position_values = p22_object_columns.drop(['preferred_foot','work_rate'], axis = 1)
p22_object_columns.drop(p22_position_values.columns, axis = 1, inplace=True)

In [137]:
p22_position_values

,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,89+3,89+3,89+3,92,93,93,93,92,93,93,...,64+3,64+3,64+3,66+3,61+3,50+3,50+3,50+3,61+3,19+3
1,90+2,90+2,90+2,85,88,88,88,85,86+3,86+3,...,66+3,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3
2,90+1,90+1,90+1,88,89,89,89,88,86+3,86+3,...,59+3,59+3,59+3,63+3,60+3,53+3,53+3,53+3,60+3,20+3
3,83+3,83+3,83+3,90,88,88,88,90,89+2,89+2,...,63+3,63+3,63+3,67+3,62+3,50+3,50+3,50+3,62+3,20+3
4,83+3,83+3,83+3,88,87,87,87,88,89+2,89+2,...,80+3,80+3,80+3,79+3,75+3,69+3,69+3,69+3,75+3,21+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,44+2,44+2,44+2,46,45,45,45,46,46+2,46+2,...,46+2,46+2,46+2,48+2,48+2,46+2,46+2,46+2,48+2,15+2
19235,45+2,45+2,45+2,47,46,46,46,47,47+2,47+2,...,46+2,46+2,46+2,49+2,48+2,44+2,44+2,44+2,48+2,14+2
19236,45+2,45+2,45+2,46,46,46,46,46,47+2,47+2,...,46+2,46+2,46+2,47+2,47+2,45+2,45+2,45+2,47+2,12+2
19237,47+2,47+2,47+2,47,47,47,47,47,45+2,45+2,...,29+2,29+2,29+2,33+2,32+2,26+2,26+2,26+2,32+2,15+2


In [138]:
p22_position_values = p22_position_values.applymap(remove_modifiers)

# Print the modified DataFrame
p22_position_values

,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,89,89,89,92,93,93,93,92,93,93,...,64,64,64,66,61,50,50,50,61,19
1,90,90,90,85,88,88,88,85,86,86,...,66,66,66,64,61,60,60,60,61,19
2,90,90,90,88,89,89,89,88,86,86,...,59,59,59,63,60,53,53,53,60,20
3,83,83,83,90,88,88,88,90,89,89,...,63,63,63,67,62,50,50,50,62,20
4,83,83,83,88,87,87,87,88,89,89,...,80,80,80,79,75,69,69,69,75,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,44,44,44,46,45,45,45,46,46,46,...,46,46,46,48,48,46,46,46,48,15
19235,45,45,45,47,46,46,46,47,47,47,...,46,46,46,49,48,44,44,44,48,14
19236,45,45,45,46,46,46,46,46,47,47,...,46,46,46,47,47,45,45,45,47,12
19237,47,47,47,47,47,47,47,47,45,45,...,29,29,29,33,32,26,26,26,32,15


In [139]:
#Converting the position values into int type
p22_position_values = p22_position_values.astype('int64')

In [140]:
#Checking the number of null values in each column
p22_position_values.isnull().sum()

ls     0
st     0
rs     0
lw     0
lf     0
cf     0
rf     0
rw     0
lam    0
cam    0
ram    0
lm     0
lcm    0
cm     0
rcm    0
rm     0
lwb    0
ldm    0
cdm    0
rdm    0
rwb    0
lb     0
lcb    0
cb     0
rcb    0
rb     0
gk     0
dtype: int64

In [141]:
#Checking the percentage of nan values in the numeric column
nan_percentage = (p22_numeric_columns.isna().sum() / len(p22_numeric_columns)) * 100
nan_percentage

overall                         0.000000
potential                       0.000000
value_eur                       0.384635
wage_eur                        0.317064
age                             0.000000
height_cm                       0.000000
weight_kg                       0.000000
club_team_id                    0.317064
league_level                    0.317064
club_jersey_number              0.317064
club_contract_valid_until       0.317064
nationality_id                  0.000000
weak_foot                       0.000000
skill_moves                     0.000000
international_reputation        0.000000
release_clause_eur              6.112584
pace                           11.081657
shooting                       11.081657
passing                        11.081657
dribbling                      11.081657
defending                      11.081657
physic                         11.081657
attacking_crossing              0.000000
attacking_finishing             0.000000
attacking_headin

In [142]:
p22_numeric_columns.drop(['goalkeeping_speed'], axis = 1, inplace=True) #Removing because it has a high percentage of nan values

In [143]:
#Filling nan values by imputation with Mean/Median
p22_numeric_columns.fillna(p22_numeric_columns.mean(), inplace=True)

In [144]:
#Checking for nan values in the object column
p22_object_columns.isna().sum()

preferred_foot    0
work_rate         0
dtype: int64

In [145]:
#One-hot encoding for 'Preferred foot' and 'Work rate'
p22_object_columns_encoded = pd.get_dummies(p22_object_columns, p22_object_columns.columns, dtype= int)

p22_object_columns_encoded.head()

,preferred_foot_Left,preferred_foot_Right,work_rate_High/High,work_rate_High/Low,work_rate_High/Medium,work_rate_Low/High,work_rate_Low/Low,work_rate_Low/Medium,work_rate_Medium/High,work_rate_Medium/Low,work_rate_Medium/Medium
0,1,0,0,0,0,0,0,0,0,1,0
1,0,1,0,0,1,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,0,0,0
3,0,1,0,0,1,0,0,0,0,0,0
4,0,1,1,0,0,0,0,0,0,0,0


In [146]:
#The dataframe that we will be using to train our model
p22_refined = pd.concat([p22_numeric_columns, p22_object_columns_encoded,p22_position_values],axis = 1)

p22_refined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 94 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      19239 non-null  int64  
 1   potential                    19239 non-null  int64  
 2   value_eur                    19239 non-null  float64
 3   wage_eur                     19239 non-null  float64
 4   age                          19239 non-null  int64  
 5   height_cm                    19239 non-null  int64  
 6   weight_kg                    19239 non-null  int64  
 7   club_team_id                 19239 non-null  float64
 8   league_level                 19239 non-null  float64
 9   club_jersey_number           19239 non-null  float64
 10  club_contract_valid_until    19239 non-null  float64
 11  nationality_id               19239 non-null  int64  
 12  weak_foot                    19239 non-null  int64  
 13  skill_moves     

In [147]:
#Scaling the new dataframe
p22_refined_scaled = scaler.fit_transform(p22_refined)
p22_refined_scaled_df = pd.DataFrame(p22_refined_scaled, columns =p22_refined.columns)
p22_refined_scaled_df

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_jersey_number,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,3.957501,3.601780,9.889601,15.998022,1.851089,-1.646467,-0.416315,-0.929914,-0.4746,0.506404,...,0.731039,0.731039,0.731039,0.851651,0.532431,-0.168529,-0.168529,-0.168529,0.532431,-0.150436
1,3.812154,3.437470,15.350958,13.425844,1.429869,0.539166,0.856805,-0.930872,-0.4746,-0.668061,...,0.874397,0.874397,0.874397,0.707442,0.532431,0.504293,0.504293,0.504293,0.532431,-0.150436
2,3.666806,3.273160,5.546836,13.425844,2.272309,0.830584,1.139720,-0.931056,-0.4746,-0.779914,...,0.372643,0.372643,0.372643,0.635337,0.461418,0.033318,0.033318,0.033318,0.461418,-0.084537
3,3.666806,3.273160,16.601147,13.425844,0.798039,-0.917923,-0.982145,-0.929914,-0.4746,-0.612134,...,0.659360,0.659360,0.659360,0.923755,0.603443,-0.168529,-0.168529,-0.168529,0.603443,-0.084537
4,3.666806,3.273160,16.140551,17.541329,1.008649,-0.043669,-0.699230,-0.931074,-0.4746,-0.220646,...,1.877905,1.877905,1.877905,1.789011,1.526608,1.109834,1.109834,1.109834,1.526608,-0.018637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,-2.728494,-3.134932,-0.365905,-0.412474,-0.676231,-0.189378,-1.547976,1.140780,-0.4746,0.841965,...,-0.559186,-0.559186,-0.559186,-0.446232,-0.390734,-0.437658,-0.437658,-0.437658,-0.390734,-0.414034
19235,-2.728494,-1.984762,-0.360641,-0.438196,-1.308061,-0.917923,-0.699230,-0.923065,-0.4746,0.338623,...,-0.559186,-0.559186,-0.559186,-0.374128,-0.390734,-0.572223,-0.572223,-0.572223,-0.390734,-0.479934
19236,-2.728494,-2.642002,-0.361957,-0.438196,-0.886841,-0.480796,-0.416315,1.114820,-0.4746,0.562331,...,-0.559186,-0.559186,-0.559186,-0.518337,-0.461746,-0.504940,-0.504940,-0.504940,-0.461746,-0.611733
19237,-2.728494,-1.820452,-0.360641,-0.438196,-1.308061,-1.209340,-1.265061,1.114820,-0.4746,-0.500280,...,-1.777732,-1.777732,-1.777732,-1.527802,-1.526936,-1.783303,-1.783303,-1.783303,-1.526936,-0.414034


In [148]:
p22_refined_scaled_df = p22_refined_scaled_df[p21_refined_scaled_df.columns]

In [149]:
p22_refined_scaled_df

,overall,release_clause_eur,value_eur,movement_reactions,wage_eur,mentality_composure,passing,dribbling,potential,mentality_vision,age,shooting,mentality_positioning,power_stamina,movement_agility
0,3.957501,9.591752,9.889601,3.599846,15.998022,3.131025,3.548135,3.564477,3.601780,3.006228,1.851089,2.992852,2.174690,0.552196,1.850364
1,3.812154,13.244084,15.350958,3.489252,13.425844,2.473077,2.284233,2.575529,3.437470,1.980597,1.429869,2.992852,2.276621,0.799952,0.908358
2,3.666806,5.380179,5.546836,3.599846,13.425844,3.048782,2.389558,2.795295,3.273160,1.614300,2.272309,3.143798,2.276621,0.861892,1.513933
3,3.666806,16.109335,16.601147,3.046874,13.425844,2.884295,3.021509,3.454594,3.273160,2.639931,0.798039,2.313597,1.817931,1.109649,2.186795
4,3.666806,15.660560,16.140551,3.268063,17.541329,2.555321,3.758786,2.795295,3.273160,2.932969,1.008649,2.540015,1.919862,1.605162,1.042930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,-2.728494,-0.363165,-0.365905,-0.934526,-0.412474,-1.721343,-1.191498,-1.600026,-3.134932,-0.803260,-0.676231,-1.309098,-0.628416,-0.748528,-0.571938
19235,-2.728494,-0.357711,-0.360641,-1.376903,-0.438196,-0.898908,-0.770197,-1.819792,-1.984762,-0.363704,-1.308061,-1.007207,0.034136,-0.005257,0.033638
19236,-2.728494,-0.358953,-0.361957,-1.708687,-0.438196,-1.803587,-1.296823,-1.490143,-2.642002,-0.583482,-0.886841,-1.158153,-0.271657,-0.438832,-0.370079
19237,-2.728494,-0.354535,-0.360641,-1.487498,-0.438196,-0.898908,-2.244749,-1.600026,-1.820452,-1.023038,-1.308061,-0.478897,-0.169726,-0.810467,0.571927


In [150]:
#Loading the data
X = p22_refined_scaled_df.drop(['overall'], axis = 1)
y = p22_refined_scaled_df['overall']

In [156]:
from sklearn.ensemble import RandomForestRegressor
# Split the data into a training set and a test set
X_test = p22_refined_scaled_df.drop(['overall'], axis = 1)

# Make predictions on the test set
y_pred = model_gbr.predict(X_test)


In [153]:
#Performing cross-validation on the model
scores = cross_val_score(model_rfr, X, y, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Calculate the mean and standard deviation of the scores
mean_score = scores.mean()
std_score = scores.std()

print("Cross-Validation Scores: ", scores)
print("Mean Score: ", mean_score)
print("Standard Deviation: ", std_score)

KeyboardInterrupt: 

**Saving the model**

In [157]:
#From testing, we've decided to go with model_rfr, the Random Forest Regressor.
import pickle


In [158]:
with open('FIFA_Rating_Generator.pkl', 'wb') as file:
    pickle.dump(model_gbr, file)


In [159]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)